In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np 
import pandas as pd 
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torchvision import datasets, transforms

import warnings
warnings.filterwarnings('ignore')
seed = np.random.seed(100)

In [2]:
#Question 1
#Define functions
def training_loop(n_epochs, optimizer, model, loss_fn, in_train, in_val, out_train, out_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(in_train) # <1>
        loss_train = loss_fn(p_train, out_train)

        p_val = model(in_val) # <1>
        loss_val = loss_fn(p_val, out_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print(datetime.datetime.now(), f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
            
def binary_map(x):
    return x.map({'yes':1,"no":0})

In [3]:
# load the dataset
housing = pd.DataFrame(pd.read_csv("./Housing.csv")) 
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'price']
varList=['mainroad','guestroom','basement','hotwaterheating','airconditioning', 'prefarea']
input_size = len(num_vars)-1

housing[varList] = housing[varList].apply(binary_map)

scaler = StandardScaler()
# scaler = MinMaxScaler()

#Split data into training and validation sets
df_train, df_test = train_test_split(housing, train_size=0.8, test_size=0.2, random_state=seed)

df_Newtrain = df_train[num_vars]
df_Newtest = df_test[num_vars]

# scaling the data
df_Newtrain[num_vars] = scaler.fit_transform(df_Newtrain[num_vars])
df_Newtest[num_vars] = scaler.fit_transform(df_Newtest[num_vars])

#Create input and output arrays for both training and validation
out_Newtrain = df_Newtrain.pop('price')
in_Newtrain = df_Newtrain
out_Newtest = df_Newtest.pop('price')
in_Newtest = df_Newtest

# convert the data to tensors
in_train = torch.tensor(in_Newtrain.values).float()
in_val = torch.tensor(in_Newtest.values).float()
out_train = torch.tensor(out_Newtrain.values).float().unsqueeze(-1)
out_val = torch.tensor(out_Newtest.values).float().unsqueeze(-1)

In [4]:
#Question 1, part 1
seq_model = nn.Sequential(
            nn.Linear(input_size, 8), #hidden layer 1
            nn.Tanh(),
            nn.Linear(8, 1)) #output layer

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    in_train = in_train,
    in_val = in_val, 
    out_train = out_train,
    out_val = out_val)

2022-03-04 19:31:55.414809 Epoch 1, Training loss 1.0984, Validation loss 1.0486
2022-03-04 19:31:55.418088 Epoch 10, Training loss 1.0850, Validation loss 1.0331
2022-03-04 19:31:55.428843 Epoch 20, Training loss 1.0706, Validation loss 1.0164
2022-03-04 19:31:55.440536 Epoch 30, Training loss 1.0567, Validation loss 1.0003
2022-03-04 19:31:55.452157 Epoch 40, Training loss 1.0433, Validation loss 0.9847
2022-03-04 19:31:55.463712 Epoch 50, Training loss 1.0303, Validation loss 0.9697
2022-03-04 19:31:55.473631 Epoch 60, Training loss 1.0178, Validation loss 0.9551
2022-03-04 19:31:55.484770 Epoch 70, Training loss 1.0056, Validation loss 0.9409
2022-03-04 19:31:55.489877 Epoch 80, Training loss 0.9939, Validation loss 0.9271
2022-03-04 19:31:55.499696 Epoch 90, Training loss 0.9824, Validation loss 0.9138
2022-03-04 19:31:55.507637 Epoch 100, Training loss 0.9713, Validation loss 0.9008
2022-03-04 19:31:55.527026 Epoch 110, Training loss 0.9606, Validation loss 0.8882
2022-03-04 19:3

In [5]:
#Question 1, part 2
seq_model_2 = nn.Sequential(
            nn.Linear(input_size, 8), #hidden layer 1
            nn.Tanh(),
            nn.Linear(8, 4), #hidden layer 2
            nn.Tanh(),
            nn.Linear(4, 2), #hidden layer 3
            nn.Tanh(),
            nn.Linear(2, 1)) #output layer


optimizer = optim.SGD(seq_model_2.parameters(), lr=1e-3)

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model_2,
    loss_fn = nn.MSELoss(),
    in_train = in_train,
    in_val = in_val, 
    out_train = out_train,
    out_val = out_val)

2022-03-04 19:31:55.618988 Epoch 1, Training loss 1.4355, Validation loss 1.4379
2022-03-04 19:31:55.628790 Epoch 10, Training loss 1.4025, Validation loss 1.4044
2022-03-04 19:31:55.639816 Epoch 20, Training loss 1.3683, Validation loss 1.3695
2022-03-04 19:31:55.649982 Epoch 30, Training loss 1.3364, Validation loss 1.3369
2022-03-04 19:31:55.658346 Epoch 40, Training loss 1.3065, Validation loss 1.3065
2022-03-04 19:31:55.668024 Epoch 50, Training loss 1.2786, Validation loss 1.2780
2022-03-04 19:31:55.670087 Epoch 60, Training loss 1.2525, Validation loss 1.2513
2022-03-04 19:31:55.690871 Epoch 70, Training loss 1.2280, Validation loss 1.2263
2022-03-04 19:31:55.692748 Epoch 80, Training loss 1.2050, Validation loss 1.2028
2022-03-04 19:31:55.701586 Epoch 90, Training loss 1.1833, Validation loss 1.1806
2022-03-04 19:31:55.709986 Epoch 100, Training loss 1.1629, Validation loss 1.1598
2022-03-04 19:31:55.719288 Epoch 110, Training loss 1.1437, Validation loss 1.1401
2022-03-04 19:3

In [6]:
#Question 2:

#Load and preprocess dataset
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_train = [(img, label) for img, label in cifar10]
cifar10_test = [(img, label) for img, label in cifar10_val]

#Use GPU if available
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

Files already downloaded and verified
Files already downloaded and verified
cuda


In [7]:
#Question 2, Part 1:

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)

model_cifar = nn.Sequential(
            nn.Linear(3072, 512), #Hidden Layer 1
            nn.Tanh(),
            nn.Linear(512, 10)) #Output layer

model_cifar.to(device)

learning_rate = 1e-2

optimizer = optim.SGD(model_cifar.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 301

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
      print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch, loss)) 

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Train Accuracy: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Val Accuracy: %f" % (correct / total))

2022-03-04 19:32:20.922285 Epoch 0, Training loss 2.089669942855835
2022-03-04 19:32:35.645967 Epoch 10, Training loss 1.3846532106399536
2022-03-04 19:32:50.611277 Epoch 20, Training loss 1.3014476299285889
2022-03-04 19:33:05.442218 Epoch 30, Training loss 0.8343853950500488
2022-03-04 19:33:20.414386 Epoch 40, Training loss 0.2873496115207672
2022-03-04 19:33:35.621919 Epoch 50, Training loss 0.5012293457984924
2022-03-04 19:33:50.506790 Epoch 60, Training loss 0.18483269214630127
2022-03-04 19:34:05.415690 Epoch 70, Training loss 0.14726722240447998
2022-03-04 19:34:20.309935 Epoch 80, Training loss 0.0986277312040329
2022-03-04 19:34:35.028396 Epoch 90, Training loss 0.05771343410015106
2022-03-04 19:34:49.656886 Epoch 100, Training loss 0.04853050038218498
2022-03-04 19:35:04.702568 Epoch 110, Training loss 0.02474435791373253
2022-03-04 19:35:19.528246 Epoch 120, Training loss 0.01835634745657444
2022-03-04 19:35:34.231601 Epoch 130, Training loss 0.027338389307260513
2022-03-04

In [8]:
#Question 2, Part 1:

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)

model_cifar_2 = nn.Sequential(
            nn.Linear(3072, 512), #hidden layer 1
            nn.Tanh(),
            nn.Linear(512, 1024), #hidden layer 2
            nn.Tanh(),
            nn.Linear(1024, 512), #hidden layer 3
            nn.Tanh(),
            nn.Linear(512, 10)) #output layer

model_cifar_2.to(device)

learning_rate = 1e-2

optimizer = optim.SGD(model_cifar_2.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 301

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar_2(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
      print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch, loss))

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar_2(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Train Accuracy: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar_2(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Val Accuracy: %f" % (correct / total))

2022-03-04 19:39:57.107777 Epoch 0, Training loss 1.5523165464401245
2022-03-04 19:40:23.170262 Epoch 10, Training loss 1.2748993635177612
2022-03-04 19:40:51.795532 Epoch 20, Training loss 1.4034696817398071
2022-03-04 19:41:14.767318 Epoch 30, Training loss 0.42979735136032104
2022-03-04 19:41:37.978402 Epoch 40, Training loss 0.3505880832672119
2022-03-04 19:42:02.356666 Epoch 50, Training loss 0.01225757785141468
2022-03-04 19:42:34.338985 Epoch 60, Training loss 0.004762476775795221
2022-03-04 19:42:57.891761 Epoch 70, Training loss 0.004877153318375349
2022-03-04 19:43:20.841858 Epoch 80, Training loss 0.0021790186874568462
2022-03-04 19:43:43.703571 Epoch 90, Training loss 0.0009847992332652211
2022-03-04 19:44:06.697824 Epoch 100, Training loss 0.000773354375269264
2022-03-04 19:44:33.467547 Epoch 110, Training loss 0.0007180912070907652
2022-03-04 19:44:57.090393 Epoch 120, Training loss 0.00029932305915281177
2022-03-04 19:45:23.988738 Epoch 130, Training loss 0.0007916414178